In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from siamese.data import SiameseDataSource
from siamese.model import (
    resnet_baseline, ModelRunner, LossCallback)

import collections
from catalyst.dl.callbacks import (
    Logger, TensorboardLogger, MetricCallback, LRFinder,
    OptimizerCallback, SchedulerCallback, CheckpointCallback, OneCycleLR)
from sklearn import metrics

In [3]:
torch.cuda.is_available()

False

In [3]:
TRAIN_FOLDER = "/home/ivb/nvme/data/raw/train"
TRAIN_CSV = "/home/ivb/nvme/data/train.csv"

INFER_FOLDER = "/home/ivb/nvme/data/raw/test"

In [4]:
loaders = SiameseDataSource.prepare_loaders(
    mode="train",
    n_workers=8,
    batch_size=512,
    train_folder=TRAIN_FOLDER,
    train_csv=TRAIN_CSV,
    train_folds=[1, 2, 3, 4], 
    valid_folds=[5],
    #infer_folder=INFER_FOLDER,
)

train samples: 11264
train batches: 22
valid samples: 2560
valid batches: 5


In [5]:
model = resnet_baseline(resnet=dict(arch="resnet18", pooling="GlobalAvgPool2d"))
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-7, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)

In [6]:
# the only tricky part
n_epochs = 1000
logdir = "../siamese/logs"

callbacks = collections.OrderedDict()

callbacks["loss"] = LossCallback()
callbacks["optimizer"] = OptimizerCallback()

# OneCylce custom scheduler callback
#callbacks["scheduler"] = OneCycleLR(
#     cycle_len=20,
#     div=3, cut_div=4, momentum_range=(0.95, 0.85))
callbacks["scheduler"] = LRFinder(final_lr=10, n_steps=1000)

# Pytorch scheduler callback
callbacks["accuracy"] = MetricCallback(
    "accuracy", 
    metric_fn=lambda o, t: metrics.accuracy_score(t.view(-1).detach().cpu().numpy(), (o.view(-1).detach().cpu().numpy() > 0).astype(int) )
)
callbacks["saver"] = CheckpointCallback()
callbacks["logger"] = Logger()
callbacks["tflogger"] = TensorboardLogger()

In [7]:
runner = ModelRunner(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler)

In [ ]:
runner.train(
    loaders=loaders, 
    callbacks=callbacks, 
    logdir=logdir,
    epochs=1000, verbose=True)

0 * Epoch (train):   9% 2/22 [00:42<07:44, 23.22s/it, accuracy=0.49219, base/batch_time=15.99426, base/data_time=0.06129, base/sample_per_second=32.01148, loss=0.73329, lr=0.00000, momentum=0.90000]